In [1]:
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler

In [2]:
featdir = Path('audio_features')
embdir = Path('embeddings')
weightsdir = Path('weights')
lbldir = Path('labels')

In [3]:
NFEATURES = 207
# here we tried reducing the dimensionality a bit, but the clustering is much
# more difficult. So we keep the match 207 features --> 207-D embedding for now.
NEMBEDD = 207

## Load Features

In [4]:
tr_audio = np.load(featdir / 'tr_audio_features.npy')[:,:NFEATURES]
tst_audio = np.load(featdir / 'tst_audio_features.npy')[:,:NFEATURES]
val_audio = np.load(featdir / 'val_audio_features.npy')[:,:NFEATURES]

ss = StandardScaler()
tr_audio = ss.fit_transform(tr_audio)
tst_audio = ss.transform(tst_audio)
val_audio = ss.transform(val_audio)

tr_lbl_audio = np.load(lbldir / 'tr_lbl.npy')
tst_lbl_audio = np.load(lbldir / 'tst_lbl.npy')
val_lbl_audio = np.load(lbldir / 'val_lbl.npy')

tr_onehot_audio = to_categorical(tr_lbl_audio)
tst_onehot_audio = to_categorical(tst_lbl_audio)
val_onehot_audio = to_categorical(val_lbl_audio)

## Define Dense Clusterer

In [5]:
def get_clusterer():
    model = tf.keras.models.Sequential()

    model.add(Dense(NEMBEDD, input_shape=(NEMBEDD,), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(NEMBEDD, activation = 'relu'))

    return model

In [6]:
clusterer = get_clusterer()
print(clusterer.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 207)               43056     
                                                                 
 dropout (Dropout)           (None, 207)               0         
                                                                 
 dense_1 (Dense)             (None, 207)               43056     
                                                                 
Total params: 86,112
Trainable params: 86,112
Non-trainable params: 0
_________________________________________________________________
None


2022-05-31 14:21:13.405829: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Training

In [7]:
trainclust = False
if trainclust:
    import tensorflow_addons as tfa
    loss = tfa.losses.TripletSemiHardLoss(margin=2.0)
    clusterer.compile(
        optimizer=tf.keras.optimizers.Adam(0.005),
        loss=loss)
    
    train = (tr_audio, tr_lbl_audio)
    val = (val_audio, val_lbl_audio)
    
    history = clusterer.fit(
        train[0], train[1],
        validation_data=val,
        batch_size=500,
        epochs=10)
else:
    clusterer.load_weights(weightsdir / 'audio_clusterer_trimmed.h5')

## Compute Embeddings from Dense Clusterer

In [8]:
tr_out = clusterer(tr_audio)
val_out = clusterer(val_audio)
tst_out = clusterer(tst_audio)

np.save(embdir / 'embeddings_val_audio.npy', val_out)
np.save(embdir / 'embeddings_test_audio.npy', tst_out)
np.save(embdir / 'embeddings_train_audio.npy', tr_out)

## Classifying the Embeddings using a DenseNet

In [9]:
train = (tr_out, tr_onehot_audio)
val = (val_out, val_onehot_audio)

def get_classifier():
    model = tf.keras.models.Sequential()
    
    model.add(Dense(NEMBEDD, input_shape=(NEMBEDD,), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, input_shape=(NEMBEDD,), activation = 'relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, input_shape=(NEMBEDD,), activation = 'relu'))
    model.add(Dropout(0.7))
    model.add(Dense(49, activation = 'softmax'))

    return model

classifier = get_classifier()

#### Training

In [10]:
loss = tf.keras.losses.CategoricalCrossentropy()
trainclass = True
if trainclass:
    # Compile the model
    classifier.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=loss,
        metrics=['accuracy'])
    
    history = classifier.fit(
        train[0], train[1],
        validation_data=val,
        batch_size=1000,
        epochs=400)

else:
    classifier.load_weights(weightsdir / 'audio_classifier.h5')

Epoch 1/400
3/3 [==============================] - 1s 83ms/step - loss: 10.1572 - accuracy: 0.0193 - val_loss: 4.0460 - val_accuracy: 0.0168
Epoch 2/400
3/3 [==============================] - 0s 26ms/step - loss: 7.2043 - accuracy: 0.0218 - val_loss: 3.8091 - val_accuracy: 0.0284
Epoch 3/400
3/3 [==============================] - 0s 26ms/step - loss: 5.6684 - accuracy: 0.0298 - val_loss: 3.7480 - val_accuracy: 0.1095
Epoch 4/400
3/3 [==============================] - 0s 27ms/step - loss: 4.7909 - accuracy: 0.0340 - val_loss: 3.7370 - val_accuracy: 0.1705
Epoch 5/400
3/3 [==============================] - 0s 26ms/step - loss: 4.3893 - accuracy: 0.0365 - val_loss: 3.7371 - val_accuracy: 0.2042
Epoch 6/400
3/3 [==============================] - 0s 22ms/step - loss: 4.1245 - accuracy: 0.0463 - val_loss: 3.7363 - val_accuracy: 0.2347
Epoch 7/400
3/3 [==============================] - 0s 19ms/step - loss: 3.9459 - accuracy: 0.0565 - val_loss: 3.7290 - val_accuracy: 0.2695
Epoch 8/400
3/3 [==

## Testing

In [11]:
pred = np.argmax(classifier(tst_out), axis=1)
print(f"Test accuracy : {np.sum(pred==tst_lbl_audio)/pred.size*100:.01f}%")

Test accuracy : 77.4%
